In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd
import json


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
#크롬 브라우저 열기
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='dkdud1871@naver.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'ayoung0112!'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)


In [3]:
def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

In [4]:
def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
time.sleep(10)

In [5]:
tag_list =[]
comments =[]
comment_more_btn="button.dCJp8.afkep"
comment_ids_objects_css="ul.Mr508 > div.ZyFrc > li.gElp9.rUo9f > div.P9YgZ > div.C7I1f > div.C4VMK > h3" 
comment_texts_objects_css="ul.Mr508 > div.ZyFrc > li.gElp9.rUo9f > div.P9YgZ > div.C7I1f > div.C4VMK > span" 



def get_content(driver):
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    try:
        while True:
            try:
                more_btn = driver.find_element_by_css_selector(comment_more_btn)
                more_btn.click()
            except:
                break
    except:
        print("----------------------fail to click more btn----------------------------------") 
        #continue

    try:
        comment_data= {}
        comment_ids_objects =driver.find_elements_by_css_selector(comment_ids_objects_css)
        comment_texts_objects = driver.find_elements_by_css_selector(comment_texts_objects_css)
    
        try: 
            for i in range(len(comment_ids_objects)): 
                comment_data[str((i+1))] = {"comment_id":comment_ids_objects[i].text, "comment_text":comment_texts_objects[i].text} 
        except:
            print("fail")
    except:
        comment_id =None
        comment_text = None
        comment_data ={}
        
    try:
        if comment_data != {}:
            
            keys =list(comment_data.keys())
            
            for key in keys: 
                if comment_data[key]['comment_id'] == upload_id: 
                    tags = re.findall('#[A-Za-z0-9가-힣]+', comment_data[key]['comment_text']) 
                    tag = ''.join(tags).replace("#"," ") # "#" 제거 
                    tag_data = tag.split() 
                    for tag_one in tag_data: 
                        instagram_tags.append(tag_one)

    except:
        
        None
        
    comment_json = json.dumps(comment_data) 
    comments.append(comment_json)


    data = [content,tag_data, comments]
    return data

In [6]:
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

In [11]:
word = "풀무원바른먹거리"
url = insta_searching(word)

driver.implicitly_wait(9)
driver.get(url)
time.sleep(9)

click_first(driver)

result_list=[]

target = 8
for i in range(target):
    try:
        data = get_content(driver)
        result_list.append(data)
        next_page(driver)
    except:
        time.sleep(9)
        next_page(driver)
    

In [12]:
df = pd.DataFrame(result_list)
df.columns = ['본문','해쉬태그','댓글']
df

,본문,해쉬태그,댓글
0,.[풀무원- 바른 먹거리 교육] 3-5학년 엄마들이 좋아하는 교육. 🤗영양소의 종류...,"[0428, juneha, 풀무원바른먹거리]","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."
1,💕풀무원 바른 먹거리 온라인 비대면 교육💕1차시/2월17일2차시/2월24일풀무원 바...,"[풀무원, 풀무원바른먹거리, 바른먹거리교육, 비대면교육, 초등집콕생활]","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."
2,#저녁메뉴...우리집 풀무원사랑💚..#풀무원바른먹거리#푸드머스#스틱피쉬커틀렛,"[저녁메뉴, 풀무원바른먹거리, 푸드머스, 스틱피쉬커틀렛]","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."
3,#딸기야🍓🍓🍓🍓🍓⁣⁣출동! 딸기밭으로~~~~⁣친구들이 너무나 기다렸던⁣노랑차 타는 ...,"[딸기야, 딸기밭견학, 우리밖에없어요, 오늘도, 행복미소최고]","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."
4,#친구들모여라👶🏻👧🏻🧒🏻⁣⁣행복미소 어린이집에서⁣2021년도 신입원아를 모집합니다⁣...,"[친구들모여라, 포항, 연일, 가정어린이집, 49평, 11월, 가을, 효자, 유강,...","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."
5,#풀무원 #풀무원교육 #풀무원바른먹거리 #풀무원바른먹거리교육 #바른먹거리교육 #초등...,"[풀무원, 풀무원교육, 풀무원바른먹거리, 풀무원바른먹거리교육, 바른먹거리교육, 초등...","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."
6,..풀무원 달걀농장 체험중~😘..#오한결#풀무원바른먹거리 #키자니아서울 #달걀농장체...,"[오한결, 풀무원바른먹거리, 키자니아서울, 달걀농장체험중, 친환경농부가되어보아요, ...","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."
7,#21년새학기이야기⁣⁣3월👶🏻🧒👧⁣신학기 시작한지 벌써 3주차❤⁣⁣우리 예쁜반.햇님...,"[21년새학기이야기, 바른먹거리, 행복미소어린이집, 내일은3월생일파티, 친구들아, ...","[{}, {}, {}, {}, {}, {""1"": {""comment_id"": ""han..."


In [13]:
df.to_csv("인스타_수정중중.csv", header = True, index = False, encoding='utf-8-sig')

In [14]:
driver.close()